In [1]:
import sys
sys.path.append('../../../Py/Build')
from suite import *
from random import *

### Cholesky factorization
We here use the Cholesky factorization implemented in the C++ library suite to create a function that solve a liner system by solving the upper and lower triangular linear system obtained from the Cholesky factorization.

In [4]:
A = mat(3,3)
A.from_Array([1.0,2.0,1.0,  2.0,8.0,4.0,  1.0,4.0,11.0])
print(A)
b = vec(3)
b.from_Array([1.0,2.0,3.0])
print(b)

[1.000000,2.000000,1.000000]
[2.000000,8.000000,4.000000]
[1.000000,4.000000,11.000000]

(1.000000,2.000000,3.000000)


We check that the matrix A is symmetric comparing it to A transpose.

In [3]:
print(A)
print(A.T())

[1.000000,2.000000,1.000000]
[2.000000,8.000000,4.000000]
[1.000000,4.000000,11.000000]

[1.000000,2.000000,1.000000]
[2.000000,8.000000,4.000000]
[1.000000,4.000000,11.000000]



The eigenvalue of A are 0.468, 5.462, 14.069 therefore the matrix is positive definite and we can perform Cholesky factorization.

In [4]:
def Solve(A,b):
    """
    Ax=b
    L(L^T)x=b
    Ly=b
    (L^T)x=y
    """
    #Init vector to store solutions of the linear systems
    y = vec(b.len())
    x = vec(b.len())
    LT = mat(b.len(),b.len())
    #We perform Cholesky factorization
    L = Cholesky(A)
    LT = L.T()
    #Init the linear system Ly=b
    S1 = LinSys(L,b)
    S1.setType("LOWER")
    #Solve linear system using forward sub
    y=S1.ForwardSub()
    #Init the linear system (L^T)x=y
    S2 = LinSys(LT,y)
    S2.setType("UPPER")
    #Solve linear system using backward sub
    x = S2.BackSub()
    #Freeing memory
    LT.free()
    L.free()
    y.free()
    return x;

In [5]:
Solve(A,b)

(1.000000,-0.111111,0.222222)

### Gram-Schmidt
We also implemented the classical Gram-Schmidt algorithm for QR decomposition to check computation made by hand suing Householder reflections.

In [6]:
A = mat(3,3)
A.from_Array([12.0,-51.0,4.0,  6.0,167.0,-68.0,  -4.0,24.0,-41.0])
A

[12.000000,-51.000000,4.000000]
[6.000000,167.000000,-68.000000]
[-4.000000,24.000000,-41.000000]

In [7]:
GS(A)

([0.857143,-0.394286,-0.331429]
 [0.428571,0.902857,0.034286]
 [-0.285714,0.171429,-0.942857],
 [14.000000,21.000000,-14.000000]
 [0.000000,175.000000,-70.000000]
 [0.000000,0.000000,35.000000])

### Parallel Cholesky

We here show the speed up obtained thanks to parallel implementation of the Cholesky-Crout algorithm using OpenMP. In particular we follow the implementation outlined in Ruschel, João Paulo Tarasconi. "Parallel implementations of the cholesky decomposition on CPUs and GPUs." (2016).

In [11]:
N = 250;
A = mat(N,N)
A.parallel=True;
#Generating sequence of random number.
Random =[]
for i in range(N**2):
    Random.append(random())
#Filling the Suite array.
A.from_Array(Random)
M = mat(N,N)
M = A*A.T()

In [12]:
%timeit Cholesky(M)

14.2 ms ± 672 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [13]:
%timeit ParallelChol(M)

8.31 ms ± 317 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
